In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self, layers):
        # layers/activations
        self.input = np.random.rand(4, 1)
        self.h1 = np.random.rand(4, 1)
        self.h2 = np.random.rand(3, 1)
        self.output = np.random.rand(3, 1)
        
        # weights 
        self.w_1 = np.random.rand(4, 4)
        self.w_2 = np.random.rand(3, 4)
        self.w_3 = np.random.rand(3, 3)
        
        # biases 
        self.b_1 = np.random.rand(4, 1)
        self.b_2 = np.random.rand(3, 1)
        self.b_3 = np.random.rand(3, 1)

    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))

    def softmax(self, activations):
        return np.exp(activations) / np.sum(np.exp(activations))
    
    def feed_forward(self, X):
        # input
        self.input = np.reshape(X, (-1,1))

        # input to h1
        h1_activations = self.sigmoid(np.dot(self.w_1, self.input) + self.b_1)
        self.h1 = h1_activations


        # h1 to h2
        h2_activations = self.sigmoid(np.dot(self.w_2, h1_activations) + self.b_2)
        self.h2 = h2_activations

        # h2 to output 
        output_activations = (np.dot(self.w_3, h2_activations) + self.b_3)
        self.output = self.softmax(output_activations)

        return (self.output, max(self.output))
    
    def back_prop(self, y, learning_rate=0.1):

        output_error = 2 * (self.output-y)
        w_3 = -learning_rate * (output_error.dot(self.h2.T))
        b_3 = -learning_rate *output_error
        
        
        h2_error = w_3.T.dot(output_error) * ((self.sigmoid(self.h2)*(1-self.sigmoid(self.h2))))
        w_2 = -learning_rate * (h2_error.dot(self.h1.T))
        b_2 = -learning_rate * h2_error
        
        h1_error = w_2.T.dot(h2_error) * ((self.sigmoid(self.h1)*(1-self.sigmoid(self.h1))))
        w_1 = -learning_rate * (h1_error.dot(self.input.T))
        b_1 = -learning_rate * h1_error
        
        self.w_1 += w_1
        self.w_2 += w_2
        self.w_3 += w_3
        
        self.b_1 += b_1
        self.b_2 += b_2
        self.b_3 += b_3
        return 0
       

    def train(self, X, y, itterations):
        y_dummies = pd.get_dummies(y)
        data = pd.concat([X, y_dummies], axis=1)
        for index, row in data.iterrows():
            y = np.reshape(row.tolist()[4:], (-1,1))
            x = row.tolist()[:4]
            output_activations, prediction = self.feed_forward(x)
            self.back_prop(output_activations, y)


        for i in range(itterations):
            current_data = data.iloc[i]
            y = np.reshape(current_data.tolist()[4:], (-1,1))
            x = current_data.tolist()[:4]
            print(x)
            print(y)
        return 0
    
    def gradient_descent(X, Y, alpha, iterations):
        for i in range(iterations):
            outputs, prediction = forward_prop(W1, b1, W2, b2, X)
            back_prop(X, Y)
            if i % 10 == 0:
                print("Iteration: ", i)
#                 predictions = get_predictions(A2)
#                 print(get_accuracy(predictions, Y))
        return 0 

In [4]:
list(X_train.iloc[0])

[5.5, 2.4, 3.7, 1.0]

In [5]:
# number of neurons at each layers input, h1, h2, output
layers = (len(X_train.iloc[0]), 4, 3, 3)

In [6]:
new_class = Network(layers=layers)

In [7]:
new_class.train(X_train, y_train, 100)

x: [5.5, 2.4, 3.7, 1.0]
y: [[0.]
 [1.]
 [0.]]
output: [[0.30485775]
 [0.40655454]
 [0.2885877 ]]

x: [4.8, 3.0, 1.4, 0.1]
y: [[1.]
 [0.]
 [0.]]
output: [[0.2203669 ]
 [0.56848987]
 [0.21114323]]

x: [5.5, 2.6, 4.4, 1.2]
y: [[0.]
 [1.]
 [0.]]
output: [[0.41325221]
 [0.3960222 ]
 [0.19072559]]

x: [5.0, 3.2, 1.2, 0.2]
y: [[1.]
 [0.]
 [0.]]
output: [[0.27975139]
 [0.56801245]
 [0.15223616]]

x: [6.9, 3.1, 5.1, 2.3]
y: [[0.]
 [0.]
 [1.]]
output: [[0.48330818]
 [0.37906295]
 [0.13762887]]

x: [5.9, 3.0, 5.1, 1.8]
y: [[0.]
 [0.]
 [1.]]
output: [[0.38177668]
 [0.32339813]
 [0.29482519]]

x: [5.0, 3.4, 1.6, 0.4]
y: [[1.]
 [0.]
 [0.]]
output: [[0.27638389]
 [0.24446629]
 [0.47914981]]

x: [6.6, 2.9, 4.6, 1.3]
y: [[0.]
 [1.]
 [0.]]
output: [[0.4678347 ]
 [0.20105605]
 [0.33110924]]

x: [5.7, 2.8, 4.1, 1.3]
y: [[0.]
 [1.]
 [0.]]
output: [[0.3459654 ]
 [0.38297527]
 [0.27105933]]

x: [5.0, 2.3, 3.3, 1.0]
y: [[0.]
 [1.]
 [0.]]
output: [[0.24359678]
 [0.55457145]
 [0.20183177]]

x: [6.3, 3.4, 5.6, 2

0

In [8]:
c

NameError: name 'c' is not defined

In [ ]:
p